In [1]:
import numpy as np
import itertools
import pygame
import random
import copy
import sys

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

# Colors
LIGHT_BROWN = (181, 101, 29)
LIGHT_GRAY = (200, 200, 200)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLACK = (0, 0, 0)

# Grid size
TILE_SIZE = 20

# Room size
ROOM_WIDTH = 20  # tiles
ROOM_HEIGHT = 10  # tiles
BED_WIDTH = 6

pygame 2.1.2 (SDL 2.0.18, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Room:
    def __init__(self, x, y, side="LEFT"):
        self.x = x
        self.y = y
        if side=="LEFT":
            list_beds = [(x + 1, y + ROOM_HEIGHT - 4), (x + BED_WIDTH + 3, y + ROOM_HEIGHT - 4)]
            list_desks = [(x + 1, y + 1), (x + BED_WIDTH + 3, y + 1)]
        else:
            list_beds = [(x + 4, y + ROOM_HEIGHT - 4), (x + ROOM_WIDTH - BED_WIDTH - 1, y + ROOM_HEIGHT - 4)]
            list_desks = [(x + 4, y + 1), (x + ROOM_WIDTH - BED_WIDTH - 1, y + 1)]
        self.beds = list_beds  # Bed positions
        self.desks = list_desks  # Desk positions

    def draw(self, screen):
        # Draw beds
        for bx, by in self.beds:
            pygame.draw.rect(screen, WHITE, (bx * TILE_SIZE, by * TILE_SIZE, TILE_SIZE * BED_WIDTH, TILE_SIZE * 3))
            pygame.draw.rect(screen, RED, (bx * TILE_SIZE, (by+0.5) * TILE_SIZE, TILE_SIZE*1.5, TILE_SIZE*2))
        # Draw desks
        for dx, dy in self.desks:
            pygame.draw.rect(screen, BLACK, (dx * TILE_SIZE, dy * TILE_SIZE, TILE_SIZE * 6, TILE_SIZE * 2))

In [3]:
class Student:
    def __init__(self, name, photo, room_number, floor_number, learned_ratio = 0, energy_ratio = 100):
        self.name = name
        self.photo = pygame.image.load(photo)
        self.room_number = room_number
        self.floor_number = floor_number
        self.learned_ratio = learned_ratio
        self.energy_ratio = energy_ratio

In [4]:
class Floor:
    def __init__(self, rows, cols, num_rooms):
        self.rows = rows
        self.cols = cols
        self.num_rooms = num_rooms
        self.rooms = []
        self.generate_rooms()

    def generate_rooms(self):
        x, y = 0, 0  # Start room placement
        rooms_left = 0
        
        for i in range(self.num_rooms):
            if y + ROOM_HEIGHT <= self.rows:
                self.rooms.append(Room(x, y))
                y += ROOM_HEIGHT-1
            else:
                rooms_left = self.num_rooms - i
                break
                
        x, y = self.cols - ROOM_WIDTH, 0
        for i in range(rooms_left):
            if y + ROOM_HEIGHT <= self.rows:
                self.rooms.append(Room(x, y, "RIGHT"))
                y += ROOM_HEIGHT-1

    def draw(self, screen):
        # Draw the floor
        for row in range(self.rows):
            for col in range(self.cols):
                color = LIGHT_BROWN if (row, col) not in self.room_walls() else LIGHT_GRAY
                pygame.draw.rect(screen, color, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE))
                pygame.draw.rect(screen, BLACK, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)

        # Draw rooms
        for room in self.rooms:
            room.draw(screen)

    def room_walls(self):
        walls = set()
        for room in self.rooms:
            x, y = room.x, room.y
            # Add the walls of the room
            for i in range(ROOM_WIDTH):
                walls.add((y, x + i))  # Top wall
                walls.add((y + ROOM_HEIGHT - 1, x + i))  # Bottom wall
            for j in range(ROOM_HEIGHT):
                walls.add((y + j, x))  # Left wall
                walls.add((y + j, x + ROOM_WIDTH - 1))  # Right wall
        return walls

In [5]:
class Dormitory:
    def __init__(self, rows, cols, num_floors, num_rooms_per_floor, students):
        self.num_floors = num_floors
        self.floors = [Floor(rows, cols, num_rooms_per_floor) for _ in range(num_floors)]
        self.students = students
        self.current_floor = 0
        self.selected_room = None  # Add this to track the selected room

    def draw(self, screen, input_buffer, input_rect, room_input_buffer, room_input_rect):
        # Draw the current floor
        self.floors[self.current_floor].draw(screen)

        # Draw students
        students_on_floor = [s for s in self.students if s.floor_number == self.current_floor]
        for student in students_on_floor:
            room = self.floors[self.current_floor].rooms[student.room_number]
            student_num = student.name.split('_')[0]
            bed_pos = room.beds[0] if student_num == 'Student1' else room.beds[1]
            student_img = pygame.transform.scale(student.photo, (TILE_SIZE * 1.5, TILE_SIZE * 1.5))
            screen.blit(student_img, (bed_pos[0] * TILE_SIZE, (room.y + ROOM_HEIGHT / 2 - 1.5) * TILE_SIZE))

        # Draw the panel for displaying the current floor
        pygame.draw.rect(screen, LIGHT_GRAY, (SCREEN_WIDTH - PANEL_WIDTH, 0, PANEL_WIDTH, SCREEN_HEIGHT))
        pygame.draw.rect(screen, BLACK, (SCREEN_WIDTH - PANEL_WIDTH, 0, PANEL_WIDTH, SCREEN_HEIGHT), 2)

        font = pygame.font.SysFont("Arial", 24)
        text_surface = font.render(f"Floor: {self.current_floor + 1}", True, BLACK)
        screen.blit(text_surface, (SCREEN_WIDTH - PANEL_WIDTH + 20, 20))

        # Draw the input field
        pygame.draw.rect(screen, WHITE, input_rect)
        pygame.draw.rect(screen, BLACK, input_rect, 2)

        input_surface = font.render(input_buffer, True, BLACK)
        input_desc = font.render(f"Go to: ", True, BLACK)
        screen.blit(input_surface, (input_rect.x + 5, input_rect.y))
        screen.blit(input_desc, (SCREEN_WIDTH - PANEL_WIDTH + 20, 60))

        # Draw the room input field
        pygame.draw.rect(screen, WHITE, room_input_rect)
        pygame.draw.rect(screen, BLACK, room_input_rect, 2)

        room_input_surface = font.render(room_input_buffer, True, BLACK)
        room_input_desc = font.render(f"Room: ", True, BLACK)
        screen.blit(room_input_surface, (room_input_rect.x + 5, room_input_rect.y))
        screen.blit(room_input_desc, (SCREEN_WIDTH - PANEL_WIDTH + 20, 105))

        # Display stats for the selected room
        font2 = pygame.font.SysFont("Arial", 20)
        if self.selected_room is not None:
            students_in_room = [s for s in self.students if s.floor_number == self.current_floor and s.room_number == self.selected_room]
            screen.blit(font2.render("Students in room:", True, BLACK), (SCREEN_WIDTH - PANEL_WIDTH + 20, 150))
            height = 180
            for i, student in enumerate(students_in_room): 
                student_info = f"{student.name}"
                screen.blit(font2.render(student_info, True, BLACK), (SCREEN_WIDTH - PANEL_WIDTH + 20, height))
                height+=30
                student_info = f"- Learned: {student.learned_ratio}%"
                screen.blit(font2.render(student_info, True, BLACK), (SCREEN_WIDTH - PANEL_WIDTH + 20, height))
                height+=30
                student_info = f"- Energy: {student.energy_ratio}%"
                screen.blit(font2.render(student_info, True, BLACK), (SCREEN_WIDTH - PANEL_WIDTH + 20, height))
                height+=60

    def switch_floor(self, floor_number):
        if 0 <= floor_number < self.num_floors:
            self.current_floor = floor_number
            self.selected_room = None  # Reset the selected room when switching floors

    def select_room(self, room_number):
        if 0 <= room_number < len(self.floors[self.current_floor].rooms):
            self.selected_room = room_number

In [ ]:
pygame.init()

ROWS, COLS = 37, 50
SCREEN_WIDTH, SCREEN_HEIGHT = COLS * TILE_SIZE + 300, ROWS * TILE_SIZE
PANEL_WIDTH = 300
num_floors = 10
num_rooms_per_floor = 8

screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Dormitory")

stickerman_photo = "stickerman2.png"
students = [
    Student(f"Student1_room{room+1}_floor{floor+1}", stickerman_photo, room, floor) for floor in range(num_floors) for room in range(num_rooms_per_floor)
]
students = students + [
    Student(f"Student2_room{room+1}_floor{floor+1}", stickerman_photo, room, floor) for floor in range(num_floors) for room in range(num_rooms_per_floor)
]

# Create the Dormitory
dormitory = Dormitory(ROWS, COLS, num_floors, num_rooms_per_floor, students)

# Input buffer for typing floor numbers
input_buffer = ""
input_rect = pygame.Rect(SCREEN_WIDTH - 120, 60, 100, 30)

room_input_buffer = ""
room_input_rect = pygame.Rect(SCREEN_WIDTH - 120, 105, 100, 30)

# Main game loop
clock = pygame.time.Clock()
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                running = False
            elif event.key == pygame.K_RETURN:  # Confirm floor or room change
                if input_buffer.isdigit():
                    floor_number = int(input_buffer) - 1  # Floors are 1-indexed for user
                    dormitory.switch_floor(floor_number)
                    input_buffer = ""  # Clear the buffer
                elif room_input_buffer.isdigit():
                    room_number = int(room_input_buffer) - 1  # Rooms are 1-indexed for user
                    dormitory.select_room(room_number)
                    room_input_buffer = ""  # Clear the buffer
            elif event.key == pygame.K_BACKSPACE:
                if input_rect.collidepoint(pygame.mouse.get_pos()):
                    input_buffer = input_buffer[:-1]
                elif room_input_rect.collidepoint(pygame.mouse.get_pos()):
                    room_input_buffer = room_input_buffer[:-1]
            else:
                if event.unicode.isdigit():
                    if input_rect.collidepoint(pygame.mouse.get_pos()):
                        input_buffer += event.unicode
                    elif room_input_rect.collidepoint(pygame.mouse.get_pos()):
                        room_input_buffer += event.unicode

    # Clear the screen
    screen.fill(BLACK)

    # Draw the current floor and UI
    dormitory.draw(screen, input_buffer, input_rect, room_input_buffer, room_input_rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()